In [6]:
from quant.sconce import sconce
from models import VGG
import torch
import torch.nn as nn
import torch.nn.functional as F
from dataset import dataloader
import torch.optim as optim


In [7]:

vgg = VGG()
vgg.load_state_dict(torch.load("vgg.cifar.pretrained.pth"))

<All keys matched successfully>

In [11]:

sconces = sconce()
sconces.model= vgg # Model Definition
sconces.criterion = nn.CrossEntropyLoss() # Loss
sconces.optimizer= optim.Adam(sconces.model.parameters(), lr=1e-4)
sconces.scheduler = optim.lr_scheduler.CosineAnnealingLR(sconces.optimizer, T_max=200)
sconces.dataloader = dataloader
sconces.epochs = 5 #Number of time we iterate over the data
sconces.device  = 'cuda'
sconces.experiment_name = "vgg-qat"
print(sconces.evaluate())

93.13627254509018


In [12]:
model_int8, model_fp32_trained = sconces.qat()


 
==================== Quantization-Aware Training(QAT) ====================


Epoch:1 Train Loss: 0.00000 Validation Accuracy: 93.10621


Epoch:2 Train Loss: 0.00000 Validation Accuracy: 93.01603


Epoch:3 Train Loss: 0.00000 Validation Accuracy: 93.13627


Epoch:4 Train Loss: 0.00000 Validation Accuracy: 93.12625


Epoch:5 Train Loss: 0.00000 Validation Accuracy: 92.97595


In [13]:
model_int8

GraphModule(
  (backbone): Module(
    (conv0): QuantizedConvReLU2d(3, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.04432808235287666, zero_point=0, padding=(1, 1))
    (conv1): QuantizedConvReLU2d(64, 128, kernel_size=(3, 3), stride=(1, 1), scale=0.026911167427897453, zero_point=0, padding=(1, 1))
    (pool0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): QuantizedConvReLU2d(128, 256, kernel_size=(3, 3), stride=(1, 1), scale=0.014782778918743134, zero_point=0, padding=(1, 1))
    (conv3): QuantizedConvReLU2d(256, 256, kernel_size=(3, 3), stride=(1, 1), scale=0.016939718276262283, zero_point=0, padding=(1, 1))
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv4): QuantizedConvReLU2d(256, 512, kernel_size=(3, 3), stride=(1, 1), scale=0.015096738934516907, zero_point=0, padding=(1, 1))
    (conv5): QuantizedConvReLU2d(512, 512, kernel_size=(3, 3), stride=(1, 1), scale=0.013277239166200161, zero_poi

In [21]:
model_int8.backbone.conv0.weight()

tensor([[[[ 0.1097,  0.1978,  0.1817],
          [-0.0450, -0.0144, -0.0809],
          [-0.0629, -0.0665, -0.1025]],

         [[ 0.1169,  0.2230,  0.2104],
          [-0.0773, -0.0270, -0.1133],
          [-0.1924, -0.1852, -0.1852]],

         [[ 0.1097,  0.2284,  0.1960],
          [-0.0144, -0.0036, -0.0054],
          [-0.1781, -0.1277, -0.1727]]],


        [[[-0.0288,  0.3023,  0.3657],
          [ 0.1152,  0.1670,  0.1209],
          [ 0.0029,  0.0317, -0.1382]],

         [[-0.1555, -0.0230,  0.0230],
          [ 0.0029, -0.1699, -0.1814],
          [ 0.1267, -0.0864, -0.1699]],

         [[-0.1526, -0.1325, -0.1440],
          [-0.1152, -0.2995, -0.2361],
          [ 0.0086, -0.0489, -0.1641]]],


        [[[-0.5869, -0.8190, -0.2386],
          [-0.0774, -0.1548,  0.2451],
          [ 0.3225,  0.5224,  0.2967]],

         [[ 0.1161, -0.0193,  0.3934],
          [ 0.1741,  0.2902,  0.4708],
          [ 0.0258,  0.2451, -0.1161]],

         [[ 0.4063,  0.1741,  0.5030],
     

In [27]:

model_int8.backbone.conv0.__dict__

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_pre_hooks': OrderedDict(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_hooks_with_kwargs': OrderedDict(),
 '_forward_hooks_always_called': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_forward_pre_hooks_with_kwargs': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_post_hooks': OrderedDict(),
 '_modules': OrderedDict(),
 'in_channels': 3,
 'out_channels': 64,
 'kernel_size': (3, 3),
 'stride': (1, 1),
 'padding': (1, 1),
 'dilation': (1, 1),
 'transposed': False,
 'output_padding': (0, 0),
 'groups': 1,
 'padding_mode': 'zeros',
 '_packed_params': <torch.ScriptObject object at 0x2308dad0>,
 'scale': 0.04432808235287666,
 'zero_point': 0}

In [ ]:
scale=0.04432808235287666

In [22]:
sum([0.0018, 0.0029, 0.0064, 0.0018, 0.0108, 0.0026, 0.0117, 0.0018, 0.0025,
        0.0015, 0.0066, 0.0015, 0.0056, 0.0042, 0.0034, 0.0011, 0.0127, 0.0042,
        0.0040, 0.0028, 0.0135, 0.0042, 0.0024, 0.0077, 0.0026, 0.0024, 0.0020,
        0.0085, 0.0016, 0.0018, 0.0022, 0.0014, 0.0026, 0.0013, 0.0042, 0.0024,
        0.0029, 0.0126, 0.0027, 0.0091, 0.0011, 0.0024, 0.0027, 0.0062, 0.0032,
        0.0021, 0.0088, 0.0056, 0.0042, 0.0044, 0.0207, 0.0032, 0.0004, 0.0073,
        0.0028, 0.0162, 0.0040, 0.0038, 0.0014, 0.0017, 0.0062, 0.0024, 0.0052,
        0.0010])

0.29500000000000004